In [55]:
from pathlib import Path
from PIL import Image
import cv2
import numpy as np

img_path = "/media/ko/MAUI63/2025-11-21-rotorua-test-flight/2025-11-21/cameras/r28/10151121/_28R5117.JPG"


In [45]:
img = Image.open(img_path)


In [60]:
def variance_of_laplacian(img):
    """Calculate the variance of the Laplacian to measure blur"""
    return cv2.Laplacian(img, cv2.CV_64F).var()


def generate_blur_detection_overlay(image, window_size=100, stride=50, overlay_alpha=0.5, colormap=cv2.COLORMAP_HOT):
    rgb = np.array(image)
    h, w = rgb.shape[:2]

    # Convert to grayscale for blur analysis
    img_gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)

    # Apply Laplacian to entire image at once
    laplacian = cv2.Laplacian(img_gray, cv2.CV_64F)
    laplacian_squared = laplacian**2

    # Create blur map using local variance with convolution
    kernel = np.ones((window_size, window_size), np.float32) / (window_size * window_size)
    local_mean = cv2.filter2D(laplacian_squared, -1, kernel)

    # Downsample to match stride
    blur_map = local_mean[::stride, ::stride]

    # Use fixed scale for comparison between images (no normalization)
    # Set a reasonable upper bound for blur values - adjust this based on your data
    max_blur_value = 100  # Adjust this threshold based on your typical blur range

    # Map blur values so that LOW blur values (blurry areas) get HIGH colormap values (red)
    # Invert the mapping: (max_value - blur_value) so low blur -> high color intensity
    blur_for_colormap = max_blur_value - np.clip(blur_map, 0, max_blur_value)
    # Clip to 0.5 of the colormap range to stay in black->red portion (avoid yellow)
    blur_normalized = (blur_for_colormap / max_blur_value * 127.5).astype(np.uint8)  # 127.5 = 255 * 0.5

    # Apply OpenCV colormap
    blur_colored = cv2.applyColorMap(blur_normalized, colormap)
    # Convert from BGR to RGB (OpenCV uses BGR)
    blur_colored_rgb = cv2.cvtColor(blur_colored, cv2.COLOR_BGR2RGB)

    # Add colormap to top right as overlay:
    h_cm, w_cm = blur_colored_rgb.shape[:2]
    rgb[0:h_cm, w - w_cm : w, :] = blur_colored_rgb

    return rgb, blur_map


# Test the function with our current image - try different colormaps
overlay_result, blur_map = generate_blur_detection_overlay(
    img, window_size=20, stride=10, overlay_alpha=0.6, colormap=cv2.COLORMAP_HOT
)

print(f"Blur map shape: {blur_map.shape}")
print(f"Blur values range: {np.min(blur_map):.2f} to {np.max(blur_map):.2f}")
print(f"Mean blur value: {np.mean(blur_map):.2f}")

cv2.imwrite("blur_overlay_result.jpg", cv2.cvtColor(overlay_result, cv2.COLOR_RGB2BGR))


Blur map shape: (634, 951)
Blur values range: -0.00 to 4007.22
Mean blur value: 238.91


True

In [29]:
cv2.imwrite("blur_detection_overlay.png", cv2.cvtColor(overlay_result, cv2.COLOR_RGB2BGR))

True

In [63]:
import json
import random

survey_dir = Path("/media/ko/MAUI63/2025-11-21-rotorua-test-flight")
with open("../survey-review-app/data/2025-11-21-rotorua-test-flight/frames.json") as f:
    frames = json.load(f)

frames = [f for f in frames if f["agl"] > 500]
frames[0]

sample_size = 20
ODIR = "../.tmp/blur_analysis"

for cam_dir in (survey_dir / "2025-11-21" / "cameras").iterdir():
    cam = cam_dir.name.lower()
    output_dir = Path(ODIR) / cam
    output_dir.mkdir(exist_ok=True, parents=True)
    imgs = [
        f["images"][cam]["path"] for f in frames if cam in f["images"] and f["images"][cam]["shutter_speed"] <= 1 / 3200
    ]
    print(cam, len(imgs))
    sample = random.sample(imgs, sample_size)
    for img_file in sample:
        img = Image.open(survey_dir / img_file)
        overlay_result, blur_map = generate_blur_detection_overlay(
            img, window_size=20, stride=10, overlay_alpha=0.6, colormap=cv2.COLORMAP_HOT
        )
        output_path = output_dir / Path(img_file).name
        cv2.imwrite(str(output_path), cv2.cvtColor(overlay_result, cv2.COLOR_RGB2BGR))

l09 269
l28 275
l28 275
r09 261
r09 261
r28 283
r28 283
